In [10]:
#Make Kenpom the standard team names
import pandas as pd
import numpy as np

KP = pd.read_csv('step01_kenpom0823.csv')
repair = pd.read_csv('step04_repair.csv')
LF = list(repair['tofix'])
LR = list(repair['replacewith'])
KP['Team'] = KP['Team'].replace(LF,LR)
KP = KP[KP['Team']!='out']
sn = KP[KP['Year']==2023].sort_values('Team')['Team'].unique()
KPN = KP.sort_values('Team')['Team'].unique()
KP2fix = list(set(KPN) - set(sn))
KP.to_csv('step01_kenpom0823.csv',index=False)
KP2fix

[]

In [11]:
# Make ESPN BPI Name match Kep Pom
BPI = pd.read_csv('step02_espnbpi0823.csv')
BPI['Team'] = BPI['Team'].replace({'State':'St.'}, regex=True)
BPI['Team'] = BPI['Team'].replace(LF,LR)
BPI = BPI[BPI['Team']!='out']
BPIN = BPI['Team'].unique()
BPI2fix = list(set(BPIN) - set(sn))

BPI.to_csv('step02_espnbpi0823.csv',index=False)
BPI2fix

[]

In [12]:
# Make Baskebal Reference Team Names match Ken Pom
BR = pd.read_csv("step03_br0823.csv")
BR['Team'] = BR['Team'].replace({'State':'St.'}, regex=True)
BR['Team'] = BR['Team'].replace(LF,LR)
BR = BR[BR['Team']!='out']
BRN = BR['Team'].unique()
BR2fix = list(set(BRN) - set(sn))

BR.to_csv('step03_br0823.csv',index=False)
BR2fix

[]

In [13]:
import pandas as pd
# Make the history AGorite team names match Ken Pom
AG = pd.read_csv("step04_FUHistory.csv")
AG['AFTeam'] = AG['AFTeam'].replace(LF,LR)
AG = AG[AG['AFTeam']!='out']
AGN = AG['AFTeam'].unique()
AG2fix = list(set(AGN) - set(sn))
AG2fix

[]

In [14]:
# Make the history Underdog team names match Ken Pom
AG['AUTeam'] = AG['AUTeam'].replace(LF,LR)
AG = AG[AG['AUTeam']!='out']
AGN = AG['AUTeam'].unique()
AG2fix = list(set(AGN) - set(sn))

AG.to_csv('step04_FUHistory.csv',index=False)
AG2fix


[]

In [15]:
# Compute Seed History
AG = pd.read_csv('step04_FUHistory.csv')
LG = AG[AG['Round']=='6']
CS = pd.DataFrame({'Round','Seed'})
for x in range(0,36):
    if LG.iloc[x]["AFScore"]>LG.iloc[x]["AUScore"]:
        ws = LG.iloc[x]["AFSeed"]
    else:
        ws = LG.iloc[x]["AUSeed"]
    CS.loc[x,'Round'] = 'W'
    CS.loc[x,'Seed'] = ws
cs = pd.crosstab(index=CS['Seed'],columns=CS['Round'])
fs = pd.crosstab(index=AG['AFSeed'],columns=AG['Round'])
us = pd.crosstab(index=AG['AUSeed'],columns=AG['Round'])
ufs = fs.add(us, fill_value=0).astype(int).drop(['PI'],axis=1)
df = ufs.add(cs, fill_value=0).fillna(0).astype(int)
nos = df.max().max()
df['Exp Wins'] = (df.sum(axis=1)-nos)/nos
df.to_csv('step04_SeedHistory.csv',index=False) 

In [16]:
#Build PASE

# table of wins
fw = pd.crosstab(index=AG['AFTeam'],columns=AG['Year'])
uw = pd.crosstab(index=AG['AUTeam'],columns=AG['Year'])
cw = fw.add(uw, fill_value=0).fillna(0).astype(int) - 1
cw = cw.replace(-1, '')

# table of seeds
AG1 = AG[AG['Round']=='1']
fs = AG1.pivot(index='AFTeam', columns='Year', values = 'AFSeed').fillna(0).astype(int)
us = AG1.pivot(index='AUTeam', columns='Year', values = 'AUSeed').fillna(0).astype(int)
cs = fs.add(us, fill_value=0).fillna(0).astype(int)
cs = cs.replace(0, '')

# table of expected wins
sh =pd.read_csv('step04_SeedHistory.csv')
ew = pd.DataFrame({'Seed':np.arange(1,17),'Exp Wins':sh['Exp Wins']})

ewty = cs.replace(list(ew['Seed']),list(ew['Exp Wins']))

ewz = (ewty * -1).replace(r'^\s*$', np.nan, regex=True).fillna(0)
cwz = cw.replace(r'^\s*$', np.nan, regex=True).fillna(0)
pase = cwz.add(ewz).replace(0, np.nan)
pase['PASE']=pase.mean(axis=1)
pase = pase.iloc[:,-1:]

pase.to_csv('step04_PASE.csv')   


ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
# Make sure PASE team names are matched to Ken Pom
pase = pd.read_csv("step04_PASE.csv")
pase.columns = ['Team','PASE']
pase['Team'] = pase['Team'].replace(LF,LR)
pase = pase[pase['Team']!='out']
paseN = pase['Team'].unique()
pase2fix = list(set(paseN) - set(sn))

#pase.to_csv('../data/d3_PASE.csv',index=False)
pase2fix

[]

In [ ]:
#Combine all the datframes
KPBPI = KP.merge(BPI,how='left', on=['Year','Team'])
KPBPIBR = KPBPI.merge(BR, how = 'left', on=['Year','Team'])
KPBPIBRP = KPBPIBR.merge(pase, how = 'left', on = ['Team'])
KPBPIBRP.to_csv('step04_AllStats.csv',index=False)

In [ ]:
#Attach stats to Tournament Games
AG = pd.read_csv('step04_FUHistory.csv')
AG = AG[AG['Year']>=2008]
AGstats = AG.merge(KPBPIBRP, left_on = ['Year','AFTeam'], right_on = ['Year','Team'], how = 'left')
AGstatsandU = AGstats.merge(KPBPIBRP, left_on = ['Year','AUTeam'], right_on = ['Year','Team'], how = 'left')
AGstatsandU = AGstatsandU.drop(['Team_x','Team_y'],axis=1)
AGstatsandU.to_csv('step04_FUStats.csv',index=False)

In [ ]:
AGstatsandU

,Year,Round,Region,Game,AFSeed,AFTeam,AFScore,AUSeed,AUTeam,AUScore,...,W_y,L_y,Pts_y,Opp_y,MOV_y,SOS_y,OSRS_y,DSRS_y,SRS_y,PASE_y
0,2008,PI,PI,-1,16,Mount St. Mary's,69,16,Coppin St.,60,...,16.0,21.0,59.8,66.0,-6.24,-6.20,-12.38,-2.13,-14.51,0.231982
1,2008,1,NW,1,1,North Carolina,113,16,Mount St. Mary's,74,...,19.0,15.0,69.2,67.7,1.53,-5.12,-3.87,0.28,-3.59,0.493243
2,2008,1,NW,2,8,Indiana,72,9,Arkansas,86,...,23.0,12.0,74.1,68.3,5.74,6.73,6.67,5.81,12.48,0.281634
3,2008,1,NW,3,5,Notre Dame,68,12,George Mason,50,...,23.0,11.0,69.3,62.5,6.82,-0.75,0.96,5.11,6.07,0.448198
4,2008,1,NW,4,4,Washington St.,71,13,Winthrop,40,...,22.0,12.0,64.9,58.5,6.32,-1.45,-7.17,10.78,3.61,-0.120120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,2022,4,NE,59,2,Villanova,50,5,Houston,44,...,32.0,6.0,74.8,58.8,16.08,6.47,7.94,14.60,22.55,0.194257
925,2022,4,SE,60,1,Kansas,76,10,Miami FL,50,...,26.0,11.0,73.9,70.4,3.59,7.62,8.18,3.04,11.22,-0.164865
926,2022,5,F4,61,2,Duke,77,8,North Carolina,81,...,29.0,10.0,77.9,71.3,6.62,8.89,12.35,3.15,15.50,0.398450
927,2022,5,F4,62,1,Kansas,81,2,Villanova,65,...,30.0,8.0,71.7,62.7,9.03,10.29,5.86,13.45,19.31,0.176757
